# 02 — Optimizers (SGD vs Adam vs RMSprop)
**Goal:** Train a tiny MLP on a synthetic 2-class dataset using different optimizers; compare loss/accuracy.


In [ ]:
import torch, torch.nn as nn, torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
torch.manual_seed(0)

In [ ]:
# Data
X, y = make_moons(n_samples=2000, noise=0.2, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(),
            nn.Linear(16, 16), nn.ReLU(),
            nn.Linear(16, 2)
        )
    def forward(self, x): return self.net(x)

def train_eval(optim_name='SGD', lr=0.05, epochs=50):
    model = MLP()
    if optim_name=='SGD':
        opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optim_name=='Adam':
        opt = optim.Adam(model.parameters(), lr=lr)
    else:
        opt = optim.RMSprop(model.parameters(), lr=lr, momentum=0.9)
    crit = nn.CrossEntropyLoss()
    losses = []
    for ep in range(epochs):
        opt.zero_grad()
        out = model(X_train)
        loss = crit(out, y_train)
        loss.backward()
        opt.step()
        losses.append(loss.item())
    with torch.no_grad():
        pred = model(X_test).argmax(1)
        acc = accuracy_score(y_test.numpy(), pred.numpy())
    return losses, acc

results = {}
for name in ['SGD','Adam','RMSprop']:
    losses, acc = train_eval(name, lr=0.01, epochs=100)
    results[name] = (losses, acc)
    print(f"{name}: test acc = {acc:.3f}")

In [ ]:
# Plot losses
for name,(losses,acc) in results.items():
    plt.plot(losses, label=f'{name} (acc={acc:.2f})')
plt.legend(); plt.title('Optimizer Loss Comparison'); plt.xlabel('epoch'); plt.ylabel('loss'); plt.show()

## 🧩 Student Practice 1 — Add Adagrad
- Implement and compare **Adagrad**; tune `lr`.


In [ ]:
# TODO: Add Adagrad optimizer case and plot its curve
# opt = optim.Adagrad(model.parameters(), lr=___)


## ⚙️ Student Practice 2 — Sensitivity to Learning Rate
- For each optimizer, try `lr in {0.1, 0.01, 0.001}` and compare curves/accuracy.

In [ ]:
# TODO: Grid search small set of learning rates per optimizer and visualize
# for lr in [0.1, 0.01, 0.001]: ...
